<a href="https://colab.research.google.com/github/anthony-rio/Portfolio/blob/main/CFBD_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load API

In [ ]:
from __future__ import print_function
import time
!pip install cfbd
import cfbd
from cfbd.rest import ApiException
from pprint import pprint

# Configure API key authorization: ApiKeyAuth
configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = "MLGdS5bu8LkhDXQJlau+/k27lI1wATrpJaxw78MRh7aiB0C+Mgx5PK6KETNlebHD"
configuration.api_key_prefix['Authorization'] = 'Bearer'

In [ ]:
import pandas as pd
import numpy as np

!pip install requests beautifulsoup4 pandas
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta


In [ ]:
api_instance_players = cfbd.PlayersApi(cfbd.ApiClient(configuration))
api_instance_teams = cfbd.TeamsApi(cfbd.ApiClient(configuration))
api_instance_recruiting = cfbd.RecruitingApi(cfbd.ApiClient(configuration))
api_instance_metrics = cfbd.MetricsApi(cfbd.ApiClient(configuration))
api_instance_ratings = cfbd.RatingsApi(cfbd.ApiClient(configuration))
api_instance_betting = cfbd.BettingApi(cfbd.ApiClient(configuration))
api_instance_games = cfbd.GamesApi(cfbd.ApiClient(configuration))
api_instance_stats = cfbd.StatsApi(cfbd.ApiClient(configuration))
api_instance_plays = cfbd.PlaysApi(cfbd.ApiClient(configuration))
api_instance_venues = cfbd.VenuesApi(cfbd.ApiClient(configuration))


In [ ]:
start_year = 2015
this_year = 2024
num_weks = 15
all_years = range(start_year, this_year + 1)
current_year_week = 7

Get FBS games only

In [ ]:
#get all games
every_game = []
for year in all_years:
  if year != 2024:
    every_game.extend(api_instance_games.get_games(year=year, division='fbs'))
  else:
    for i in range(current_year_week):
      every_game.extend(api_instance_games.get_games(year=year, division='fbs', week=i+1))

Historical Season Variables & Game context variables

Team longitude, latitude, and elevation

In [ ]:
fbs_schools_by_year = []
fbs_schools_year = []
fbs_long = []
fbs_lat = []
fbs_elevation = []

for year in all_years:
  api_response = api_instance_teams.get_fbs_teams(year=year)
  for i in range(len(api_response)):
      fbs_schools_by_year.append(year)
      fbs_schools_year.append(api_response[i].school)
      fbs_long.append(api_response[i].location.longitude)
      fbs_lat.append(api_response[i].location.latitude)
      fbs_elevation.append(api_response[i].location.elevation)

yearly_fbs_schools = pd.DataFrame({
    'Year': fbs_schools_by_year,
    'School': fbs_schools_year,
    'Longitude': fbs_long,
    'Latitude': fbs_lat,
    'Elevation': fbs_elevation
})

Schedule of weeks of season

In [ ]:
years = range(start_year, this_year)
schedule = pd.DataFrame({
    'Week': list(range(1, 15)) * len(years) + list(range(1, current_year_week + 1)),
    'Season': [year for year in years for _ in range(1, 15)] + [2024] * current_year_week
})

#adjust for COVID year
schedule.loc[schedule['Season'] == 2020, 'Week'] += 2

Get lists with school names, talent/recruiting metrics, returning production,

In [ ]:

# Initialize lists for storing results
ret_prod_usage = []
ret_prod_total_rushing_ppa = []
ret_prod_total_ppa = []
ret_prod_total_passing_ppa = []
ret_prod_total_receiving_ppa = []
talent = []
school_names = []
talent_year = []
ret_prod_school = []
ret_prod_year = []


# Loop through each year
for year_test in all_years:
    api_response_talent = api_instance_teams.get_talent(year=year_test)
    for i in range(len(api_response_talent)):
      school_names.append(api_response_talent[i].school)
      talent.append(api_response_talent[i].talent)
      talent_year.append(year_test)

    api_response_ret_prod = api_instance_players.get_returning_production(year=year_test)
    for i in range(len(api_response_ret_prod)):
      ret_prod_usage.append(api_response_ret_prod[i].usage)
      ret_prod_total_rushing_ppa.append(api_response_ret_prod[i].total_rushing_ppa)
      ret_prod_total_ppa.append(api_response_ret_prod[i].total_ppa)
      ret_prod_total_passing_ppa.append(api_response_ret_prod[i].total_passing_ppa)
      ret_prod_total_receiving_ppa.append(api_response_ret_prod[i].total_receiving_ppa)
      ret_prod_school.append(api_response_ret_prod[i].team)
      ret_prod_year.append(api_response_ret_prod[i].season)

# Create a DataFrame from the lists
talent_df = pd.DataFrame({
    'Talent': talent,
    'School': school_names,
    'Season': talent_year
})


talent_ret_data = pd.DataFrame({
    'School': ret_prod_school,
    'Season': ret_prod_year,
    'Ret Usage': ret_prod_usage,
    'Ret Total Rushing PPA': ret_prod_total_rushing_ppa,
    'Ret Total PPA': ret_prod_total_ppa,
    'Ret Total Passing PPA': ret_prod_total_passing_ppa,
    'Ret Total Receiving PPA': ret_prod_total_receiving_ppa
})

all_talent_data = talent_ret_data.merge(talent_df, on=['School', 'Season'], how='left')

Net Portal Blue Chips

In [ ]:
net_portal_blue_chips = []
net_portal_hts = []
for each_year in all_years:
  api_response_portal = api_instance_players.get_transfer_portal(year=each_year)
  for i in range(len(all_talent_data[(all_talent_data['Season'] == each_year)]['School'])):
    school = all_talent_data[(all_talent_data['Season'] == each_year)]['School'].iloc[i]
    net_b = 0
    net_t = 0
    filtered_data = [
      entry for entry in api_response_portal
        if entry.destination == school or entry.origin == school
    ]

    for entry in filtered_data:
      if entry is not None:  # Check if entry is not None
        if entry.rating is not None:  # Ensure rating is not None
            if entry.rating >= 0.90:
                if entry.destination == school:
                    net_b += 1  # Add 1 for destination
                if entry.origin == school:
                    net_b -= 1  # Subtract 1 for origin
            elif 0.87 < entry.rating < 0.90:  # Proper range check for rating
                if entry.destination == school:
                    net_t += 0.5  # Add 0.5 for destination
                if entry.origin == school:
                    net_t -= 0.5  # Subtract 0.5 for origin

    net_portal_blue_chips.append(net_b)
    net_portal_hts.append(net_t)

all_talent_data['Net Portal Blue Chips'] = net_portal_blue_chips
all_talent_data['Net Portal HTS'] = net_portal_hts

Load All Data

In [ ]:
every_line = pd.read_csv('every_line.csv')
elo_data = pd.read_csv('elo_data.csv')
sos_df = pd.read_csv('sos_data.csv')
all_qb_ppa_data = pd.read_csv('all_qb_ppa_data.csv')
all_advanced_stats = pd.read_csv('advanced_stats_team.csv')

Byes by team

In [ ]:

byes = schedule.merge(yearly_fbs_schools[['School', 'Year']], how='cross')
byes = byes[byes['Year'] == byes['Season']]
byes.drop(columns=['Year'], inplace=True)
byes['Bye'] = 1
byes = byes.reset_index(drop=True)

In [ ]:
for i in range(len(byes)):
  week_bye = byes.loc[i, 'Week']
  season_bye = byes.loc[i, 'Season']
  team_bye = byes.loc[i, 'School']
  test = every_line[
    ((every_line['Home Team'] == team_bye) &
     (every_line['Season'] == season_bye) &
      (every_line['Week'] == week_bye)) |
    ((every_line['Away Team'] == team_bye) &
     (every_line['Season'] == season_bye) &
     (every_line['Week'] == week_bye))
]
  if len(test) > 0:
    byes.loc[i, 'Bye'] = 0

# Shift the Bye column to the next week
byes['Bye'] = byes.groupby('School')['Bye'].shift(1)

byes.loc[byes['Week'] == 1, 'Bye'] = 0

# Display the updated DataFrame
byes.head(5)

,Week,Season,School,Bye
0,1,2015,Air Force,0.0
1,1,2015,Akron,0.0
2,1,2015,Alabama,0.0
3,1,2015,App State,0.0
4,1,2015,Arizona,0.0


All ratings related metrics

All Lines

In [ ]:
all_lines = []
point_diff = []
pd_week = []
pd_home_team = []
pd_away_team = []
pd_season = []
gametime = []

for i in range(len(schedule)):
  api_response = api_instance_betting.get_lines(year=int(schedule['Season'][i]), week=int(schedule['Week'][i]))
  fbs_schools = yearly_fbs_schools[yearly_fbs_schools['Year'] == schedule['Season'][i]]['School'].tolist()
  for j in range(len(api_response)):
    if api_response[j].home_team in fbs_schools and api_response[j].away_team in fbs_schools:
      if len(api_response[j].lines) == 0:
        continue
      else:
        all_lines.append(api_response[j].lines[-1].spread)
        pd_week.append(schedule['Week'][i])
        pd_home_team.append(api_response[j].home_team)
        pd_away_team.append(api_response[j].away_team)
        pd_season.append(schedule['Season'][i])
        timestamp = api_response[j].start_date

        dt = datetime.fromisoformat(timestamp[:-1])  # Remove the 'Z' and parse
        hour = dt.hour - 4  # Convert to EST
        if hour < 0:
          hour += 24
        minute = dt.minute
        decimal_time = hour + (minute / 60)
        gametime.append(decimal_time)
      if i == len(schedule['Week']) - 1:
        point_diff.append(None)
      else:
        point_diff.append(api_response[j].away_score - api_response[j].home_score)

every_line = pd.DataFrame({
    'Point Diff': point_diff,
    'Line': all_lines,
    'Week': pd_week,
    'Home Team': pd_home_team,
    'Away Team': pd_away_team,
    'Season': pd_season,
    'Game Time': gametime
})

csv_file_path = 'every_line.csv'
every_line.to_csv(csv_file_path, index=False)
every_line = pd.read_csv('every_line.csv')

Elo

In [ ]:
# import time
elo = []
elo_year = []
elo_week = []
elo_school = []


for i in range(len(schedule['Week'])):
  # fbs_schools = yearly_fbs_schools[yearly_fbs_schools['Year'] == schedule['Season'][i]]['School'].tolist()
  api_response_elo = api_instance_ratings.get_elo_ratings(year=int(schedule['Season'][i]), week=int(schedule['Week'][i]))
  for j in range(len(api_response_elo)):
    if api_response_elo:
        elo.append(api_response_elo[j].elo)
        elo_school.append(api_response_elo[j].team)
        elo_year.append(api_response_elo[j].year)
        elo_week.append(schedule['Week'][i])

elo_data = pd.DataFrame({
    'School': elo_school,
    'Season': elo_year,
    'Week': elo_week,
    'Elo': elo
})

csv_file_path = 'elo_data.csv'
elo_data.to_csv(csv_file_path, index=False)
elo_data = pd.read_csv('elo_data.csv')


SOS

In [ ]:
dates = []
# Function to generate the next 12 dates incrementing by 7 days
def generate_dates(start_date, weeks=14):
    return [(start_date + timedelta(weeks=i)).strftime('%Y-%m-%d') for i in range(weeks)]

# Define the start date
start_dates = [datetime(2015, 9, 7), datetime(2016, 9, 5), datetime(2017, 9, 4), datetime(2018, 9, 3), datetime(2019, 9, 2),
               datetime(2020, 9, 21), datetime(2021, 9, 6), datetime(2022, 9, 5), datetime(2023, 9, 4), datetime(2024, 9, 2)] # Starting date
for i in range(len(start_dates) - 1):
  dates.extend(generate_dates(start_dates[i]))
dates.extend(generate_dates(start_dates[-1], weeks=current_year_week))

# List to hold the scraped data
school_data = []
sos_data = []
sos_year = []
sos_week = []

In [ ]:
import time
# Loop through each date
for i in range(len(dates) - 1):

  url = f"https://www.teamrankings.com/college-football/ranking/schedule-strength-by-other?date={dates[i]}"
  response = requests.get(url)

  # Check if the request was successful
  if response.status_code == 200:
      soup = BeautifulSoup(response.text, 'html.parser')

      # Use the selector gadget's CSS class to find the relevant data
      rows_school = soup.select('.nowrap')
      rows_sos = soup.select('.nowrap+ .text-right')

      # Extract text from each row and add to the data list
      for row in rows_school:
          school_data.append(row.text.strip())
      for row in rows_sos:
          sos_data.append(row.text.strip())
      sos_year.extend([schedule['Season'][i]] * len(rows_school))
      if schedule['Season'][i] == 2020:
        sos_week.extend([schedule['Week'][i] + 3] * len(rows_school))
      else:
        sos_week.extend([schedule['Week'][i] + 1] * len(rows_school))


# Convert the data to a DataFrame for better manipulation
sos_df = pd.DataFrame({
    'School': school_data,
    'SOS': sos_data,
    'Week': sos_week,
    'Season': sos_year
})

#fix names
sos_df['School'] = sos_df['School'].str.extract(r'^(.*?)\s*\(\d')

csv_file_path = 'sos_data.csv'
sos_df.to_csv(csv_file_path, index=False)
sos_df = pd.read_csv('sos_data.csv')

In [ ]:
sos_df

,School,SOS,Week,Season
0,Wisconsin,25.2,2,2015
1,Texas,23.8,2,2015
2,Ohio St,23.6,2,2015
3,Arkansas St,22.5,2,2015
4,GA Southern,20.8,2,2015
...,...,...,...,...
16937,Kent St,-15.1,7,2024
16938,Central Mich,-15.3,7,2024
16939,LA Tech,-15.6,7,2024
16940,N Mex State,-16.5,7,2024


QB Data

In [ ]:
cum_qb_ppa_data = pd.DataFrame(columns=['School', 'Season', 'Week', 'QB Avg PPA'])
all_qb_ppa_data = pd.DataFrame(columns=['School', 'Season', 'Week', 'QB Avg PPA'])

# Iterate through the schedule
for i in range(len(schedule['Week'])):
  qb_ppa_data = pd.DataFrame(columns=['School', 'Season', 'Week', 'QB Avg PPA'])
  week = int(schedule['Week'][i])
  year = int(schedule['Season'][i])
  fbs_schools = yearly_fbs_schools[yearly_fbs_schools['Year'] == schedule['Season'][i]]['School'].tolist()
  # Fetch API response for the current week, year, and school
  api_response_test = api_instance_metrics.get_player_game_ppa(year=year, week=week, position='QB', exclude_garbage_time=True, threshold=10)
  for j in range(len(api_response_test)):
    if (api_response_test[j].team in fbs_schools) & (api_response_test[j].opponent in fbs_schools):
      qb_ppa_data.loc[len(qb_ppa_data)] = [api_response_test[j].team, api_response_test[j].season, api_response_test[j].week, api_response_test[j].average_ppa.all]
  qb_ppa_data = qb_ppa_data.loc[qb_ppa_data.groupby(['School', 'Season', 'Week'])['QB Avg PPA'].idxmax()]
  cum_qb_ppa_data = pd.concat([cum_qb_ppa_data, qb_ppa_data], ignore_index=True)
  filtered_data = cum_qb_ppa_data[cum_qb_ppa_data['Season'] == year]

  # Step 2: Group by 'School' and calculate the average 'QB Avg PPA'
  grouped_data = filtered_data.groupby('School', as_index=False)['QB Avg PPA'].mean()

  # Step 3: Create a new DataFrame with the same four columns
  final_qb_ppa_data = pd.DataFrame({
    'School': grouped_data['School'],
    'Season': [year] * len(grouped_data),  # Same season for all rows
    'Week': [week + 1] * len(grouped_data),  # Placeholder for Week, you can set this as needed
    'QB Avg PPA': grouped_data['QB Avg PPA']
  })
  all_qb_ppa_data = pd.concat([all_qb_ppa_data, final_qb_ppa_data], ignore_index=True)


csv_file_path = 'all_qb_ppa_data.csv'
all_qb_ppa_data.to_csv(csv_file_path, index=False)
all_qb_ppa_data = pd.read_csv('all_qb_ppa_data.csv')

<ipython-input-14-fbf60aafc8bf>:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  cum_qb_ppa_data = pd.concat([cum_qb_ppa_data, qb_ppa_data], ignore_index=True)
<ipython-input-14-fbf60aafc8bf>:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_qb_ppa_data = pd.concat([all_qb_ppa_data, final_qb_ppa_data], ignore_index=True)


Team advanced stats by game

In [ ]:
d_explosiveness = []
d_fpa_start = []
d_havoc_db = []
d_havoc_front_seven = []
d_line_yards = []
d_line_yards_total = []
d_open_field_yards = []
d_open_field_yards_total = []
d_passing_downs_explosiveness = []
d_passing_downs_ppa = []
d_passing_downs_rate = []
d_passing_downs_success_rate = []
d_passing_plays_explosiveness = []
d_passing_plays_ppa = []
d_passing_plays_rate = []
d_passing_plays_success_rate = []
d_passing_plays_total_ppa = []
d_points_per_opportunity = []
d_power_success = []
d_ppa = []
d_rushing_plays_explosiveness = []
d_rushing_plays_ppa = []
d_rushing_plays_rate = []
d_rushing_plays_success_rate = []
d_second_level_yards = []
d_second_level_yards_total = []
d_standard_downs_explosiveness = []
d_standard_downs_ppa = []
d_standard_downs_rate = []
d_standard_downs_success_rate = []
d_stuff_rate = []
d_success_rate = []
d_total_ppa = []
o_explosiveness = []
o_fpa_start = []
o_havoc_db = []
o_havoc_front_seven = []
o_line_yards = []
o_line_yards_total = []
o_open_field_yards = []
o_open_field_yards_total = []
o_passing_downs_explosiveness = []
o_passing_downs_ppa = []
o_passing_downs_rate = []
o_passing_downs_success_rate = []
o_passing_plays_explosiveness = []
o_passing_plays_ppa = []
o_passing_plays_rate = []
o_passing_plays_success_rate = []
o_passing_plays_total_ppa = []
o_points_per_opportunity = []
o_power_success = []
o_ppa = []
o_rushing_plays_explosiveness = []
o_rushing_plays_ppa = []
o_rushing_plays_rate = []
o_rushing_plays_success_rate = []
o_second_level_yards = []
o_second_level_yards_total = []
o_standard_downs_explosiveness = []
o_standard_downs_ppa = []
o_standard_downs_rate = []
o_standard_success_rate = []
o_stuff_rate = []
o_success_rate = []
o_total_ppa = []
stats_year = []
stats_week = []
stats_team = []

In [ ]:
for i in range(len(schedule['Week'])):
  week_i = int(schedule['Week'][i])
  year = int(schedule['Season'][i])
  api_response_advanced = api_instance_stats.get_advanced_team_season_stats(year=year, exclude_garbage_time=True, start_week=1, end_week=week_i)
  for j in range(len(api_response_advanced)):
    if api_response_advanced[j]:
      stats_team.append(api_response_advanced[j].team)
      d_explosiveness.append(api_response_advanced[j].defense.explosiveness)
      d_fpa_start.append(api_response_advanced[j].defense.field_position.average_start)
      d_havoc_db.append(api_response_advanced[j].defense.havoc.db)
      d_havoc_front_seven.append(api_response_advanced[j].defense.havoc.front_seven)
      d_line_yards.append(api_response_advanced[j].defense.line_yards)
      d_line_yards_total.append(api_response_advanced[j].defense.line_yards_total)
      d_open_field_yards.append(api_response_advanced[j].defense.open_field_yards)
      d_open_field_yards_total.append(api_response_advanced[j].defense.open_field_yards_total)
      d_passing_downs_explosiveness.append(api_response_advanced[j].defense.passing_downs.explosiveness)
      d_passing_downs_ppa.append(api_response_advanced[j].defense.passing_downs.ppa)
      d_passing_downs_rate.append(api_response_advanced[j].defense.passing_downs.rate)
      d_passing_downs_success_rate.append(api_response_advanced[j].defense.passing_downs.success_rate)
      d_passing_plays_explosiveness.append(api_response_advanced[j].defense.passing_plays.explosiveness)
      d_passing_plays_ppa.append(api_response_advanced[j].defense.passing_plays.ppa)
      d_passing_plays_rate.append(api_response_advanced[j].defense.passing_plays.rate)
      d_passing_plays_success_rate.append(api_response_advanced[j].defense.passing_plays.success_rate)
      d_passing_plays_total_ppa.append(api_response_advanced[j].defense.passing_plays.total_ppa)
      d_points_per_opportunity.append(api_response_advanced[j].defense.points_per_opportunity)
      d_power_success.append(api_response_advanced[j].defense.power_success)
      d_ppa.append(api_response_advanced[j].defense.ppa)
      d_rushing_plays_explosiveness.append(api_response_advanced[j].defense.rushing_plays.explosiveness)
      d_rushing_plays_ppa.append(api_response_advanced[j].defense.rushing_plays.ppa)
      d_rushing_plays_rate.append(api_response_advanced[j].defense.rushing_plays.rate)
      d_rushing_plays_success_rate.append(api_response_advanced[j].defense.rushing_plays.success_rate)
      d_second_level_yards.append(api_response_advanced[j].defense.second_level_yards)
      d_second_level_yards_total.append(api_response_advanced[j].defense.second_level_yards_total)
      d_standard_downs_explosiveness.append(api_response_advanced[j].defense.standard_downs.explosiveness)
      d_standard_downs_ppa.append(api_response_advanced[j].defense.standard_downs.ppa)
      d_standard_downs_rate.append(api_response_advanced[j].defense.standard_downs.rate)
      d_standard_downs_success_rate.append(api_response_advanced[j].defense.standard_downs.success_rate)
      d_stuff_rate.append(api_response_advanced[j].defense.stuff_rate)
      d_success_rate.append(api_response_advanced[j].defense.success_rate)
      d_total_ppa.append(api_response_advanced[j].defense.total_ppa)
      o_explosiveness.append(api_response_advanced[j].offense.explosiveness)
      o_fpa_start.append(api_response_advanced[j].offense.field_position.average_start)
      o_havoc_db.append(api_response_advanced[j].offense.havoc.db)
      o_havoc_front_seven.append(api_response_advanced[j].offense.havoc.front_seven)
      o_line_yards.append(api_response_advanced[j].offense.line_yards)
      o_line_yards_total.append(api_response_advanced[j].offense.line_yards_total)
      o_open_field_yards.append(api_response_advanced[j].offense.open_field_yards)
      o_open_field_yards_total.append(api_response_advanced[j].offense.open_field_yards_total)
      o_passing_downs_explosiveness.append(api_response_advanced[j].offense.passing_downs.explosiveness)
      o_passing_downs_ppa.append(api_response_advanced[j].offense.passing_downs.ppa)
      o_passing_downs_rate.append(api_response_advanced[j].offense.passing_downs.rate)
      o_passing_downs_success_rate.append(api_response_advanced[j].offense.passing_downs.success_rate)
      o_passing_plays_explosiveness.append(api_response_advanced[j].offense.passing_plays.explosiveness)
      o_passing_plays_ppa.append(api_response_advanced[j].offense.passing_plays.ppa)
      o_passing_plays_rate.append(api_response_advanced[j].offense.passing_plays.rate)
      o_passing_plays_success_rate.append(api_response_advanced[j].offense.passing_plays.success_rate)
      o_passing_plays_total_ppa.append(api_response_advanced[j].offense.passing_plays.total_ppa)
      o_points_per_opportunity.append(api_response_advanced[j].offense.points_per_opportunity)
      o_power_success.append(api_response_advanced[j].offense.power_success)
      o_ppa.append(api_response_advanced[j].offense.ppa)
      o_rushing_plays_explosiveness.append(api_response_advanced[j].offense.rushing_plays.explosiveness)
      o_rushing_plays_ppa.append(api_response_advanced[j].offense.rushing_plays.ppa)
      o_rushing_plays_rate.append(api_response_advanced[j].offense.rushing_plays.rate)
      o_rushing_plays_success_rate.append(api_response_advanced[j].offense.rushing_plays.success_rate)
      o_second_level_yards.append(api_response_advanced[j].offense.second_level_yards)
      o_second_level_yards_total.append(api_response_advanced[j].offense.second_level_yards_total)
      o_standard_downs_explosiveness.append(api_response_advanced[j].offense.standard_downs.explosiveness)
      o_standard_downs_ppa.append(api_response_advanced[j].offense.standard_downs.ppa)
      o_standard_downs_rate.append(api_response_advanced[j].offense.standard_downs.rate)
      o_standard_success_rate.append(api_response_advanced[j].offense.standard_downs.success_rate)
      o_stuff_rate.append(api_response_advanced[j].offense.stuff_rate)
      o_success_rate.append(api_response_advanced[j].offense.success_rate)
      o_total_ppa.append(api_response_advanced[j].offense.total_ppa)
      stats_year.append(year)
      stats_week.append(week_i+1)

all_advanced_stats = pd.DataFrame({
    'Offense Explosiveness': o_explosiveness,
    'Offense FPA Start': o_fpa_start,
    'Offense Havoc DB': o_havoc_db,
    'Offense Havoc Front Seven': o_havoc_front_seven,
    'Offense Line Yards': o_line_yards,
    'Offense Line Yards Total': o_line_yards_total,
    'Offense Open Field Yards': o_open_field_yards,
    'Offense Open Field Yards Total': o_open_field_yards_total,
    'Offense Passing Downs Explosiveness': o_passing_downs_explosiveness,
    'Offense Passing Downs PPA': o_passing_downs_ppa,
    'Offense Passing Downs Rate': o_passing_downs_rate,
    'Offense Passing Downs Success Rate': o_passing_downs_success_rate,
    'Offense Passing Plays Explosiveness': o_passing_plays_explosiveness,
    'Offense Passing Plays PPA': o_passing_plays_ppa,
    'Offense Passing Plays Rate': o_passing_plays_rate,
    'Offense Passing Plays Success Rate': o_passing_plays_success_rate,
    'Offense Passing Plays Total PPA': o_passing_plays_total_ppa,
    'Offense Points Per Opportunity': o_points_per_opportunity,
    'Offense Power Success': o_power_success,
    'Offense PPA': o_ppa,
    'Offense Rushing Plays Explosiveness': o_rushing_plays_explosiveness,
    'Offense Rushing Plays PPA': o_rushing_plays_ppa,
    'Offense Rushing Plays Rate': o_rushing_plays_rate,
    'Offense Rushing Plays Success Rate': o_rushing_plays_success_rate,
    'Offense Second Level Yards': o_second_level_yards,
    'Offense Second Level Yards Total': o_second_level_yards_total,
    'Offense Standard Downs Explosiveness': o_standard_downs_explosiveness,
    'Offense Standard Downs PPA': o_standard_downs_ppa,
    'Offense Standard Downs Rate': o_standard_downs_rate,
    'Offense Standard Success Rate': o_standard_success_rate,
    'Offense Stuff Rate': o_stuff_rate,
    'Offense Success Rate': o_success_rate,
    'Offense Total PPA': o_total_ppa,
    'Defense Explosiveness': d_explosiveness,
    'Defense FPA Start': d_fpa_start,
    'Defense Havoc DB': d_havoc_db,
    'Defense Havoc Front Seven': d_havoc_front_seven,
    'Defense Line Yards': d_line_yards,
    'Defense Line Yards Total': d_line_yards_total,
    'Defense Open Field Yards': d_open_field_yards,
    'Defense Open Field Yards Total': d_open_field_yards_total,
    'Defense Passing Downs Explosiveness': d_passing_downs_explosiveness,
    'Defense Passing Downs PPA': d_passing_downs_ppa,
    'Defense Passing Downs Rate': d_passing_downs_rate,
    'Defense Passing Downs Success Rate': d_passing_downs_success_rate,
    'Defense Passing Plays Explosiveness': d_passing_plays_explosiveness,
    'Defense Passing Plays PPA': d_passing_plays_ppa,
    'Defense Passing Plays Rate': d_passing_plays_rate,
    'Defense Passing Plays Success Rate': d_passing_plays_success_rate,
    'Defense Passing Plays Total PPA': d_passing_plays_total_ppa,
    'Defense Points Per Opportunity': d_points_per_opportunity,
    'Defense Power Success': d_power_success,
    'Defense PPA': d_ppa,
    'Defense Rushing Plays Explosiveness': d_rushing_plays_explosiveness,
    'Defense Rushing Plays PPA': d_rushing_plays_ppa,
    'Defense Rushing Plays Rate': d_rushing_plays_rate,
    'Defense Rushing Plays Success Rate': d_rushing_plays_success_rate,
    'Defense Second Level Yards': d_second_level_yards,
    'Defense Second Level Yards Total': d_second_level_yards_total,
    'Defense Standard Downs Explosiveness': d_standard_downs_explosiveness,
    'Defense Standard Downs PPA': d_standard_downs_ppa,
    'Defense Standard Downs Rate': d_standard_downs_rate,
    'Defense Standard Success Rate': d_standard_downs_success_rate,
    'Defense Stuff Rate': d_stuff_rate,
    'Defense Success Rate': d_success_rate,
    'Defense Total PPA': d_total_ppa,
    'Year': stats_year,
    'Week': stats_week,
    'Team': stats_team
  })

csv_file_path = 'advanced_stats_team.csv'
all_advanced_stats.to_csv(csv_file_path, index=False)
all_advanced_stats = pd.read_csv('advanced_stats_team.csv')


KeyboardInterrupt: 

PPA by team

Season team advanced stats

Advanced Stats Data Frame

Distance

In [ ]:
from geopy.distance import geodesic

def calculate_distance(row):
    point1 = (row['Latitude'], row['Longitude'])
    point2 = (row['Latitude_Away'], row['Longitude_Away'])
    return geodesic(point1, point2).miles


Formatting Fixes

In [ ]:
replacement_map = {
    'Florida St': 'Florida State',
    'Fresno St': 'Fresno State',
    'Middle Tenn': 'Middle Tennessee',
    'Boise St': 'Boise State',
    'Kent St': 'Kent State',
    'N Carolina': 'North Carolina',
    'GA Southern': 'Georgia Southern',
    'Texas A&M': 'Texas A&M',
    'U Mass': 'Massachusetts',
    'N Mex State': 'New Mexico State',
    'TX Christian': 'TCU',  # Texas Christian University
    'E Carolina': 'East Carolina',
    'Central Mich': 'Central Michigan',
    'Florida Intl': 'Florida International',
    'S Alabama': 'South Alabama',
    'S Carolina': 'South Carolina',
    'S Florida': 'UCF',
    'S Methodist': 'SMU',  # Southern Methodist University
    'S Mississippi': 'Southern Miss',
    'San Diego St': 'San Diego State',
    'San Jose St': 'San José State',
    'W Kentucky': 'Western Kentucky',
    'W Michigan': 'Western Michigan',
    'W Virginia': 'West Virginia',
    'Wash State': 'Washington State',
    'Kennesaw St': 'Kennesaw State',
    'Arizona St': 'Arizona State',
    'Georgia State': 'Georgia State',
    'Miss St': 'Mississippi State',
    'Ball St': 'Ball State',
    'Utah St': 'Utah State',
    'James Mad': 'James Madison',
    'VA Tech': 'Virginia Tech',
    'Michigan St': 'Michigan State',
    'Ohio St': 'Ohio State',
    'Penn St': 'Penn State',
    'N Illinois': 'Northern Illinois',
    'Jksnville St': 'Jacksonville State',
    'Coastal Car': 'Coastal Carolina',
    'Boston Col': 'Boston College',
    'Louisiana-Monroe': 'UL Monroe',
    'GA Tech': 'Georgia Tech',
    'Connecticut': 'UConn',
    'Bowling Grn': 'Bowling Green',
    'Arkansas St': 'Arkansas State',
    'Texas St': 'Texas State',
    'Texas A&M University': 'Texas A&M',
    'TX El Paso': 'UTEP',
    'TX San Antonio': 'UTSA',
    'Hawaii': "Hawai'i",
    'E Michigan': 'Eastern Michigan',
    'Fla Atlantic': 'Florida Atlantic',
    'Oklahoma St': 'Oklahoma State',
    'Iowa St': 'Iowa State',
    'LA Tech': 'Louisiana Tech',
    'USF': 'South Florida',
    'Colorado St': 'Colorado State',
    'Georgia St': 'Georgia State',
    'Kansas St': 'Kansas State',
    'Miss State': 'Mississippi State',
    'Oregon St': 'Oregon State',
    'Mississippi': 'Ole Miss',
    'North Carolina State': 'NC State',
    'Sam Hous St': 'Sam Houston'
}

# Add other mappings for abbreviations
for school in yearly_fbs_schools['School'].unique():
    # Add a mapping from abbreviations to full names
    # You may need to handle similar abbreviations differently
    short_name = school.replace(' ', '').replace('.', '')  # Remove spaces and periods
    replacement_map[short_name] = school

# Replace values in the DataFrame using the mapping
sos_df['School'] = sos_df['School'].replace(replacement_map)


Expand talent_df, talent_ret_data, and school_info

In [ ]:
# Function to expand each DataFrame
def expand_dataframe(df, weeks):
    # Create a new DataFrame by repeating each row for each week
    expanded_df = pd.DataFrame()
    for week in weeks:
        week_data = df.copy()
        week_data['Week'] = week
        expanded_df = pd.concat([expanded_df, week_data], ignore_index=True)
    return expanded_df

weeks = schedule['Week'].tolist()

# Expand the DataFrames
expanded_talent_df = expand_dataframe(all_talent_data, weeks)
expanded_school_info = expand_dataframe(yearly_fbs_schools, weeks)

In [ ]:
every_line
elo_data
sos_df
all_qb_ppa_data
all_advanced_stats
byes
# yearly_fbs_schools
# all_talent_data

In [ ]:
all_advanced_stats['Season'] = all_advanced_stats['Year']
all_advanced_stats.drop(columns=['Year'], inplace=True)
all_advanced_stats['School'] = all_advanced_stats['Team']
all_advanced_stats.drop(columns=['Team'], inplace=True)
expanded_school_info['Season'] = expanded_school_info['Year']
expanded_school_info.drop(columns=['Year'], inplace=True)
merged_data = expanded_school_info.merge(expanded_talent_df, on = ['School', 'Season', 'Week'], how='left')
merged_data = merged_data.drop_duplicates(subset=['School', 'Season', 'Week'], ignore_index=True)
merged_data = merged_data.merge(elo_data, on=['School', 'Season', 'Week'], how='left')
merged_data = merged_data.merge(byes, on=['School', 'Season', 'Week'], how='left')
merged_data = merged_data.merge(sos_df, on=['School', 'Season', 'Week'], how='left')
merged_data = merged_data.merge(all_qb_ppa_data, on=['School', 'Season', 'Week'], how='left')
merged_data = merged_data.merge(all_advanced_stats, on=['School', 'Season', 'Week'], how='left')



Merge Data with Every Line

In [ ]:
home_merge = pd.merge(
    every_line,
    merged_data,
    left_on=['Home Team', 'Season', 'Week'],
    right_on=['School', 'Season', 'Week'],
    suffixes=('', '_Home')
)

# Merge for Away Team
full_merged_data = pd.merge(
    home_merge,
    merged_data,
    left_on=['Away Team', 'Season', 'Week'],
    right_on=['School', 'Season', 'Week'],
    suffixes=('', '_Away')
)

full_merged_data['Distance'] = full_merged_data.apply(calculate_distance, axis=1)

# Optionally drop the 'School' columns from merged_data as they're redundant
full_merged_data = full_merged_data.drop(columns=['School', 'School_Away', 'Elevation_Away', 'Latitude', 'Longitude', 'Latitude_Away', 'Longitude_Away'])

# drop duplicates
full_merged_data = full_merged_data.drop_duplicates(subset=['Home Team', 'Away Team', 'Week', 'Season'])

# Check the resulting DataFrame
full_merged_data.head()

,Point Diff,Line,Week,Home Team,Away Team,Season,Game Time,Elevation,Ret Usage,Ret Total Rushing PPA,...,Defense Second Level Yards_Away,Defense Second Level Yards Total_Away,Defense Standard Downs Explosiveness_Away,Defense Standard Downs PPA_Away,Defense Standard Downs Rate_Away,Defense Standard Success Rate_Away,Defense Stuff Rate_Away,Defense Success Rate_Away,Defense Total PPA_Away,Distance
0,-8.0,-16.0,1,Northern Illinois,UNLV,2015,19.5,2.674530e+02,0.668,112.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1466.606158
1,-44.0,-38.0,1,West Virginia,Georgia Southern,2015,19.5,3.274097e+02,0.421,49.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,509.419011
2,17.0,7.0,1,Idaho,Ohio,2015,21.0,7.944795e+02,0.584,39.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1823.948995
3,30.0,13.5,1,Tulane,Duke,2015,21.5,1.180000e-38,0.781,31.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,770.690730
4,-8.0,-2.5,1,Hawai'i,Colorado,2015,1.0,2.000000e+01,0.778,22.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3329.817993


Save CSV of Full Merged Data

In [ ]:
csv_file_path = 'full_merged_data.csv'
full_merged_data.to_csv(csv_file_path, index=False)
full_merged_data = pd.read_csv('full_merged_data.csv')

<ipython-input-124-a90a547495e7>:3: DtypeWarning: Columns (18,96) have mixed types. Specify dtype option on import or set low_memory=False.
  full_merged_data = pd.read_csv('full_merged_data.csv')


Take out issue teams and week 1

Model